ETL de los reviews de google

Importamos las librerias

In [12]:
import pandas as pd
import os
from datetime import datetime
import re

Leer el archivo

In [2]:
# Directorio que contiene los archivos JSON
directorio = "D:/Datasets_proyecto/reviews-estados"

# Lista para almacenar los DataFrames individuales
dfs = []

# Iterar sobre cada archivo en el directorio
for filename in os.listdir(directorio):
    if filename.endswith(".json"):
        filepath = os.path.join(directorio, filename)
        
        # Leer el archivo JSON en un DataFrame
        df = pd.read_json(filepath, lines=True)
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
# Concatenar todos los DataFrames en uno solo
dfreviewsGoogle = pd.concat(dfs, ignore_index=True)

Filtramos los id para que solo sean los de ulta beauty

In [3]:
#Leemos el dataset de dfbusinessGoogle porque no viene el nombre del local , solo el ID
dfbusinessGoogle=pd.read_csv(r'D:\Datasets_proyecto\dfbusinessGoogle.csv')


# Obtén los business_id únicos de df_google
business_ids_to_keep = dfbusinessGoogle['business_id'].unique()

# Filtra df_reviews_ulta_beauty utilizando isin
dfreviewsGoogle = dfreviewsGoogle[dfreviewsGoogle['gmap_id'].isin(business_ids_to_keep)]

Borramos la columna name porque es el nombre de usuario y solo nos interesa su id , pics y resp también se van a borrar

In [4]:
dfreviewsGoogle.drop(columns=['name','pics','resp'],inplace=True)

De la columna time se va a extraer el mes , año y hora

In [5]:
# Convertir la columna 'time' a formato de fecha y hora
dfreviewsGoogle['time'] = pd.to_datetime(dfreviewsGoogle['time'], unit='ms')

# Extraer el mes, año y hora en nuevas columnas
dfreviewsGoogle['month'] = dfreviewsGoogle['time'].dt.month
dfreviewsGoogle['year'] = dfreviewsGoogle['time'].dt.year
dfreviewsGoogle['hour'] = dfreviewsGoogle['time'].dt.hour

#Borrar la columna time
dfreviewsGoogle.drop(columns=['time'],inplace=True)

Cambiamos los nombres de las columnas para estandarizar los nombres con los datos de yelp

In [7]:
# Renombrar las columnas
dfreviewsGoogle = dfreviewsGoogle.rename(columns={'gmap_id': 'business_id', 'rating': 'stars'})

Ordenamos las columnas

In [9]:
# Reordenar las columnas
column_order = ['user_id', 'business_id', 'stars', 'text', 'month', 'year', 'hour']
dfreviewsGoogle = dfreviewsGoogle[column_order]

La columna texto lo convertimos a minusculas y quitamos los caracteres especiales

In [15]:
def limpiar_texto(texto):
    # Verificar si el valor es una cadena
    if isinstance(texto, str):
        # Convertir a minúsculas
        texto = texto.lower()
        # Eliminar caracteres especiales usando expresiones regulares
        texto = re.sub(r'[^a-z0-9\s]', '', texto)
    return texto

# Aplicar la función a la columna 'text'
dfreviewsGoogle['text'] = dfreviewsGoogle['text'].apply(limpiar_texto)

Agregamos la columnas source para identiicar de donde viene la fuente

In [17]:
dfreviewsGoogle['source']='G'

Eliminamos los valores nulos y duplicados

In [20]:
# Eliminar filas donde la columna 'text' es nula
dfreviewsGoogle = dfreviewsGoogle.dropna(subset=['text'])

# Eliminar filas duplicadas en función de todas las columnas
dfreviewsGoogle = dfreviewsGoogle.drop_duplicates()

Filtramos los años de alcance de nuestro proyecto , los años 2019 , 2020 , 2021 se obtendran del dataset original pero los años 2022 y 2023 se obtendran en la carga incremental

In [24]:
# Filtrar las filas para los años 2019, 2020 y 2021
dfreviewsGoogle = dfreviewsGoogle[(dfreviewsGoogle['year'] == 2019) | (dfreviewsGoogle['year'] == 2020) | (dfreviewsGoogle['year'] == 2021)]

In [25]:
dfreviewsGoogle.head()

,user_id,business_id,stars,text,month,year,hour,source
94018,1.018389e+20,0x5342381c6ddf30e3:0xfbe922695b89d6de,5,go to this place specifically for my foundatio...,5,2021,16,G
94019,1.113166e+20,0x5342381c6ddf30e3:0xfbe922695b89d6de,5,i recommend this place\nthey are so polite an...,4,2021,16,G
94020,1.142388e+20,0x5342381c6ddf30e3:0xfbe922695b89d6de,5,love this place friendly staff and always will...,2,2021,23,G
94021,1.079620e+20,0x5342381c6ddf30e3:0xfbe922695b89d6de,5,fantastic place for beauty queens kings,2,2021,4,G
94022,1.050090e+20,0x5342381c6ddf30e3:0xfbe922695b89d6de,5,helpful staff and lots of variety,5,2021,18,G
